[Open this notebook on Colab](https://colab.research.google.com/github/probabll/ntmi-tutorials/blob/main/Encoders.ipynb)

# Guide

* Check the entire notebook before you get started, this gives you an idea of what lies ahead.

## ILOs

After completing this lab you should be able to

* specify neural text encoders in PyTorch

## General Notes

* In this notebook you are expected to use $\LaTeX$. 
* Use python3.
* Use Torch. 
* This tutorial runs on CPU with no problem.

We will use a set of standard libraries that are often used in machine learning projects. If you are running this notebook on Google Colab, all libraries should be pre-installed. If you are running this notebook locally you will need to install some additional packages, ask your TA for help if you have problems setting up.

If you need a short introduction to PyTorch [check this tutorial](https://github.com/probabll/ntmi-tutorials/blob/main/PyTorch.ipynb).


## Table of Contents



### Topics 

* [Text Encoders](#sec:Text_Encoders)
* [Vocabulary](#sec:Vocabulary)
* [From Tokens to Vectors](#sec:From_Tokens_to_Vectors)
	* [One-Hot Encoding](#sec:One-Hot_Encoding)
	* [Word embeddings](#sec:Word_embeddings)
* [Pooling from multiple vectors](#sec:Pooling_from_multiple_vectors)
	* [Sum pooling](#sec:Sum_pooling)
	* [Average pooling](#sec:Average_pooling)
* [Mapping from one real coordinate space to another](#sec:Mapping_from_one_real_coordinate_space_to_another)
	* [Linear transformation](#sec:Linear_transformation)
	* [Nonlinear activation functions](#sec:Nonlinear_activation_functions)
* [Composing multiple vectors](#sec:Composing_multiple_vectors)
	* [Concatenation](#sec:Concatenation)
	* [Feed forward network](#sec:Feed_forward_network)
	* [Recurrent neural network encoder](#sec:Recurrent_neural_network_encoder)
	 	* [Bidirectional RNN encoder](#sec:Bidirectional_RNN_encoder)


### Table of ungraded exercises

1. [One-hot encoding](#ungraded-1)
1. [Token embedding](#ungraded-2)
1. [Sum pooling](#ungraded-3)
1. [Average pooling](#ungraded-4)
1. [Limitations of pooling](#ungraded-5)
1. [Linear transformation](#ungraded-6)
1. [Activation functions](#ungraded-7)
1. [Concatenation](#ungraded-8)
1. [FFNN](#ungraded-9)

## Setting up

In [ ]:
import random
import numpy as np
np.random.seed(42)
random.seed(42)

In [ ]:
import torch

<a name='sec:Text_Encoders'></a>
# Text Encoders

In NLP applications, we often have to *encode* a piece of text, that is, map it to one (or more) vector(s) in some real coordinate space. For example, that is the case in text classification.

In this section we will discuss standard NN building blocks and how to use them to encode a document (i.e., turn a document into features) and then map that encoding to the parameters of our choice of probability mass function (pmf). 
Whenever a neural network has parameters of its own, these are initialised in some standard way (typically at random). At initialisation, these parameters are uniformative. That is, we can use the NN, but the outputs are not optimised for any specific purpose. We will implement a training procedure later, for now, it is sufficient to focus on how to specify the NN functions and making sure we understand their inputs, their outputs and how what operations they perform. 

Throughout, we will focus on building blocks useful in the design of $C$-way text classifiers. We assume a *document* is a sequence $x=\langle w_1, \ldots, w_l \rangle$ of $l$ tokens, each token comes from a vocabulary $\mathcal V$ of $V$ tokens. The label space $\mathcal C$ of our text classifier is made of $C$ classes. Hence, our goal is to map from any given $x$ to a $C$-dimensional probability vector $\boldsymbol \pi^{(x)} \in \Delta_{C-1}$.


The rough idea is as follows:
* we convert the tokens in a document to fixed-dimensional vectors ;
* then, we map these vectors to a single vector representing the entire document (depending on how we design this operation, it may or may not discard information such as the order in which the tokens ocurred);
* finally, we map this document encoding to a vector of $C$ logits (and softmax gives us $C$ probabilities), which then is used to parameterise the Categorical pmf.

In order to maximise the benefit of working with computation graphs, we will design our NNs to process batches of documents, where the documents in a batch may differ in length.

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F

<a name='sec:Vocabulary'></a>
# Vocabulary

The first thing we do when working with text is to map our tokens to unique 0-based integer identifiers. It does not matter which token gets which id, so long as the correspondence between tokens and ids is fixed and unique.

A vocabulary often contains some special symbols, which help us design good text encoders. 

For example, 

* After training we may encounter symbols we've never seen before, to better deal with those it's convenient to have a dedicated `unk` symbol. For example, if "otter" was never seen in training, but occurs at one point in a document test "a cute otter", we change that document to "a cute -UNK-".
* When working with batches of documents, we will use tensors of fixed dimensionality to store documents of variable length. The general ideal is to have the tensor be as long as needed to accommodate the longest document, as for the shorter documents, we extend them to the longest length using a special symbol `pad` symbol (later, whenever we encounter this symbol in a certain position, we know that that position is not really part of the document). For example, if we need to batch together "the nice cat" and "the cat", we instead batch together "the nice cat" and "the cat -PAD-".

There are various implementations of vocabulary, here's a very basic one (for most projects we need to adapt it to our needs).

In [ ]:
class Vocabulary:
    """
    Constructs a 1-to-1 map between symbols (strings) and 0-based identifiers
    """
    
    def __init__(self, reserve: list):
        """
        reserve: reserve this symbols in order
        """
        if len(reserve) != len(set(reserve)):
            raise ValueError("Every reserved symbol must be unique")
        self._reserve = tuple(reserve)
        self._sym2id = dict()
        self._symbols = []
        for sym in reserve:
            self.add(sym)
        
    def add(self, sym: str):
        """Add a symbol (if it is unique) and return its index"""
        idx = self._sym2id.get(sym, None)
        if idx is None:
            idx = len(self._symbols)
            self._symbols.append(sym)
            self._sym2id[sym] = idx
        return idx
    
    def idx(self, sym):
        """Return the index of an existing symbol"""
        idx = self._sym2id.get(sym, None)
        if idx is None:
            raise KeyError(f"Unknown symbol {sym}")
        return idx
    
    def symbol(self, idx):
        """Return the symbol associated with an index"""
        return self._symbols[idx]

    def __len__(self):
        """Vocabulary size"""
        return len(self._symbols)
    
    def items(self):
        """Items in the dictionary of symbols"""
        return self._sym2id.items()

In [ ]:
vocabulary = Vocabulary(['-PAD-', '-UNK-', '-BOS-', '-EOS-'])
len(vocabulary)

In [ ]:
for k, s in vocabulary.items():
    print(k, s)

In [ ]:
print(vocabulary.add("a"))
print(vocabulary.add("the"))
print(vocabulary.add("a")) # duplicates aren't added twice
print(vocabulary.add("nice"))
print(vocabulary.add("cat"))
print(vocabulary.add("dog"))
print(vocabulary.add("rabbit"))

In [ ]:
print(vocabulary.idx("dog"))

In [ ]:
print(vocabulary.symbol(0))

In [ ]:
for k, s in vocabulary.items():
    print(k, s)

From now on, if we talk about a "token" we mean a _token id_ (i.e., the 0-based integer that identifies that symbol uniquely).

<a name='sec:From_Tokens_to_Vectors'></a>
# From Tokens to Vectors

Neural networks are functions that take real-valued vectors as inputs and produce real-valued vectors as outputs. They are quite flexible, but not flexible enough to process _symbolic_ data without special treatment. 

In this section we discuss 2 techniques that we can use to map from a symbolic space, such as the set of known words (i.e., the vocabulary) to a real coordinate space (a real-valued vector space of fixed dimensionality). 

<a name='sec:One-Hot_Encoding'></a>
## One-Hot Encoding

If we know a _finite_ set of symbols (such as our vocabulary $\mathcal V$), and the total number of unique symbols is some number $V$, then the simplest technique to map symbols to vectors is to map each symbol $t \in \mathcal V$ to a vector $\mathbf v \in \mathbf R^V$ such that $v_t=1$ and $v_{d\neq t}=0$. 

This technique is called 'one-hot encoding' because it returns a vector whose coordinates are 0, except for one dimension (the one corresponding to the symbol of interest), which gets 1. 

Here we illustrate how to obtain one-hot encodings using `F.one_hot` from torch.

In [ ]:
# This is the 10-dimensional encoding of dog (the fourth symbol in the vocabulary)
F.one_hot(
    torch.tensor(vocabulary.idx("dog")).long(), 
    len(vocabulary) # output dimensionality
)

See that the output dimensionality is fixed, this means we can only encode up to $V$ elements:

In [ ]:
try: 
    F.one_hot(
        torch.tensor(-1).long(), 
        len(vocabulary) # output dimensionality
    )
except RuntimeError:
    print(f"Torch is 0-based, hence for a {len(vocabulary)}-dimensional space, we can encode from 0 to {len(vocabulary)-1}")
    
try: 
    F.one_hot(
        torch.tensor(len(vocabulary)).long(), 
        len(vocabulary) # output dimensionality
    )
except RuntimeError:
    print(f"Torch is 0-based, hence for a {len(vocabulary)}-dimensional space, we can encode from 0 to {len(vocabulary)-1}")

We can always work with batches of symbols, for example, a batch containing 2 documents of different length.

In [ ]:
F.one_hot(
    torch.tensor(
        [
            [vocabulary.idx("the"), vocabulary.idx("nice"), vocabulary.idx("dog")], # first document 
            [vocabulary.idx("a"), vocabulary.idx("cat"), vocabulary.idx("-PAD-")] # second document
        ]
    ).long(), # two example documents (already expressed as sequences of token ids)
    len(vocabulary) # vocabulary size (pretend we only know 10 words)
)

<a name='ungraded-1'></a> **Ungraded Exercise 1 - One-hot encoding**

1. The output of `F.one_hot` above should have dimensionality [2, 3, 10]. Can you tell why?
2. In general, if the input batch has shape `[B, L]` and our vocabulary has `V` tokens, what's the expected output size of `F.one_hot(input_batch, V)`?
3. How many trainable parameters are needed to specify a one-hot encoding function for a vocabulary of size `V`?


<details>
    <summary> <b>Click to see a solution</b> </summary>
 
1. Every token id in the input should have been converted to a 10-dimensional one-hot vector.
The input has shape [2, 3], that is, we have two sequences of length 3.
Hence, the output should be [2, 3, 10].
2. In general, for [B, L] inputs we expect [B, L, V] outputs.
3. None. The one-hot encoding function does not require trainable parameters, the output is "hard-coded" to be exactly the one-hot representation of a symbol, for each and every symbol in the vocabulary. 
    
---
    
</details>      


<a name='sec:Word_embeddings'></a>
## Word embeddings

Our next operation is a bit more interesting. It allows us to associate with each token a vector of trainable parameters. So, instead of encoding a symbol $t \in \mathcal V$ into a $V$-dimensional one-hot vector, we encode it into a $D$-dimensional vector of real-numbers. We normally refer to this operation as *embedding* the token into a $D$-dimensional space. Suppose we have a table of parameters $\mathbf E \in \mathbb R^{V \times D}$, with one $D$-dimensional row for each of the known symbols in the vocabulary $\mathcal V$. Then, for some symbol $t \in \mathcal V$, the embedding operation returns the vector $\mathbf e \in \mathbb R^D$ that corresponds to it.

Sometimes, we need to describe this operation in written form (for example, to sketch our model architecture). Here is a compact notation for it:
\begin{align}
    \mathbf e &= \mathrm{embed}_D(t; \mathbf E)
\end{align}

The subscript $_D$ indicates the output dimensionality. After `;` we have the trainable parameters of the operation.


If we had a sequence of symbols, for example, a document $w_{1:l}$, we could apply the embedding operation to each symbol in the sequence, and denote it this way:
\begin{align}
    \mathbf e_i &= \mathrm{embed}_D(w_i; \mathbf E) \quad \text{for }i=1,\ldots, l
\end{align}

Let's see how to use torch to specify an embedding layer. We will design a toy embedding layer, for a toy vocabulary:

In [ ]:
# this creates the layer with untrained parameters
toy_emb_dim = 2
toy_vocab_size = len(vocabulary)
toy_emb = nn.Embedding(
    num_embeddings=toy_vocab_size, 
    embedding_dim=toy_emb_dim, 
)
toy_emb

See that pytorch will intialise all 10 vectors for us, each 2-dimensional:

In [ ]:
print(toy_emb.weight)

For a 2-dimensional embedding layer, we can plot the embeddings:

In [ ]:
import matplotlib.pyplot as plt

if toy_emb.embedding_dim == 2:
    _ = plt.scatter(toy_emb.weight[:,0].detach().numpy(), toy_emb.weight[:,1].detach().numpy())
    for t, i in vocabulary.items(): # pretending our vocabulary is this toy example
        _ = plt.annotate(t, toy_emb.weight[i].detach().numpy())
    _ = plt.xlabel("Dimension 1")
    _ = plt.ylabel("Dimension 2")    
    _ = plt.title("Embeddings at initialisation")

The forward method of the embedding module will embed every token in a batch of token ids.
Let's test test it on a toy batch:

In [ ]:
toy_batch = torch.tensor(
    [
        [vocabulary.idx(s) for s in "nice cat nice dog".split()],
        [vocabulary.idx(s) for s in "a dog -PAD- -PAD-".split()],
        [vocabulary.idx(s) for s in "nice dog nice cat".split()],
        [vocabulary.idx(s) for s in "a nice dog -PAD-".split()]
    ]
)
toy_batch

In [ ]:
# this embeds the tokens in the sequences in the batch
e = toy_emb(toy_batch)
print(e.shape)
print(e)

It can be useful to count the number of parameters in a layer, here is some helper code:

In [ ]:
def num_parameters(torch_module):
    """A helper to count the number of parameters in a torch module"""
    return sum(np.prod(theta.shape) for theta in torch_module.parameters())

<a name='ungraded-2'></a> **Ungraded Exercise 2 - Token embedding**

Assume the input batch has shape `[B, L]`, our vocabulary has size `V` and our embedding vectors are `D`-dimensional. 

1. Once we pass the input batch through the embedding layer, what's the output shape?
2. How many trainable parameters do we need in order to specify an embedding layer?


<details>
    <summary> <b>Click to see a solution</b> </summary>
 
1. The output shape is `[B, L, D]` because each one of the token ids is mapped to a `D`-dimensional vector.
2. We need $V \times D$ trainable parameters, that is, $D$ parameters for each of the $V$ symbols in the vocabulary.
    
---
    
</details>      



<details>
    <summary> <b>Click to see a solution</b> </summary>

If you double-click the cell, you will be able to copy the code:

```python

# We can use this to verify that the output shape is correct

assert toy_emb(toy_batch).shape == toy_batch.shape + (toy_emb_dim,)

# We can use this to verify the size of the embedding layer

assert num_parameters(toy_emb) == toy_vocab_size * toy_emb_dim, "Embedding layers are built upon [V, D] matrices"

```

---
    
</details>      


<a name='sec:Pooling_from_multiple_vectors'></a>
# Pooling from multiple vectors

Sometimes we need to combine a variable number of $D$-dimensional vectors (they all have the same dimensionality) into a single $D$-dimensional vector, this is normally referred to as *pooling*.

There are different pooling operations, some with and some without trainable parameters. In this tutorial we will only cover two simple and rather common ones. 

<a name='sec:Sum_pooling'></a>
## Sum pooling 

The output is the elementwise sum of the inputs.

* Input: a collection of $l > 0$ vectors $\mathbf e_1, \ldots, \mathbf e_l$, all of the same dimensionality $D$.
* Output $\mathbf u \in \mathbb R^D$ defined as

\begin{align}
    \mathbf u &= \sum_{i=1}^l \mathbf e_i
\end{align}

That is, for each dimension $d \in [D]$, we have
\begin{align}
    u_d &=  \sum_{i=1}^l e_{i,d}
\end{align}

A *batched* implementation of this operation must give special treatment to PAD positions (as those should be ignored).

In [ ]:
def sum_pooling(input_sequences, sequence_mask):
    """
    Returns the sum of the vectors along the sequence dimension.
    
    input_sequences: [batch_size, max_length, D] a batch of sequences of D-dimensional vectors
    sequence_mask: [batch_size, max_length] indicates which positions are valid (i.e., not PAD)
        we use 1 for valid (not PAD) and 0 for PAD
    
    Output shape is [batch_size, D]
    """
    
    # here we replace padding positions by D-dimensional vectors of 0s, 
    #  this way those options won't contribute to the sum
    # [batch_size, max_length, D]    
    masked = torch.where(
        # we create an extra axis at the end of the tensor
        sequence_mask.unsqueeze(-1),  # this has shape [batch_size, max_length, 1]
        input_sequences,  # this has shape [batch_size, max_length, D]
        torch.zeros_like(input_sequences)  # this has shape [batch_size, max_length, D]
    )
    
    # we sum, along the sequence dimension (second last),
    #  the valid vectors (those that are not PAD)
    # [batch_size, D]
    return torch.sum(masked, dim=-2) 

Let's test this using our toy batch. To obtain a sequence mask, we compare the tokens in the batch to the PAD id (we are using `0` for that):

In [ ]:
toy_batch, toy_batch != vocabulary.idx("-PAD-")

In [ ]:
sum_pooling(F.one_hot(toy_batch, toy_vocab_size), toy_batch != vocabulary.idx("-PAD-"))

<a name='ungraded-3'></a> **Ungraded Exercise 3 - Sum pooling**

Suppose we have a batch of documents, with shape `[B, L]`, we turn the tokens into one-hot vectors of dimensionality `V`. Then, then we apply sum pooling to this input batch. 

1. What's the shape of the output?
2. Can we interpret the result as a bag-of-words encoding of the documents in the batch?
3. How many trainable parameters are needed to specify the sum pooling operation?


<details>
    <summary> <b>Click to see a solution</b> </summary>
 
1. The input batch has shape `[B, L]`, after one-hot encoding, we get `[B, L, V]`, since each token becomes a V-dimensional one-hot vector. Finally, sum pooling works along the sequence dimension, hence we have output shape `[B, V]`.
2. Yes, that's precisely how we can interpret the output. you can see it in the toy example above. 
3. None, the sum pooling operation is an elementwise transformation of existing vectors and it does not require any trainable parameters. 
    
---
    
</details>      



<details>
    <summary> <b>Click to see a solution</b> </summary>

If you double-click the cell, you will be able to copy the code:

```python

h = sum_pooling(F.one_hot(toy_batch, toy_vocab_size), toy_batch != vocabulary.idx("-PAD-"))
# Note how the 'step' or 'time' dimension is gone
assert h.shape == (toy_batch.shape[0], toy_vocab_size)
# In the first document h[0], "nice" occurs twice:
assert h[0, vocabulary.idx("nice")] == 2

```

---
    
</details>      


<a name='sec:Average_pooling'></a>
## Average pooling 

When the output is the elementwise average of the inputs, this is known as *average pooling*. 

* Input: a collection of $l > 0$ vectors $\mathbf e_1, \ldots, \mathbf e_l$, all of the same dimensionality $D$.
* Output $\mathbf u \in \mathbb R^D$ defined as

\begin{align}
    \mathbf u &= \frac{1}{l} \sum_{i=1}^l \mathbf e_i
\end{align}

That is, for each dimension $d \in [D]$, we have
\begin{align}
    u_d &= \frac{1}{l} \sum_{i=1}^l e_{i,d}
\end{align}

A with sum pooling, a *batched* implementation of this operation must give special treatment to PAD positions (as those should be ignored).

In [ ]:
def average_pooling(input_sequences, sequence_mask):
    """
    Returns the average encoding of each sequence.
    
    input_sequences: [batch_size, max_length, D] a batch of sequences of D-dimensional vectors
    sequence_mask: [batch_size, max_length] indicates which positions are valid (i.e., not PAD)
        we use 1 for valid (not PAD) and 0 for PAD
    
    Output shape is [batch_size, D]
    """
    
    # here we replace padding positions by D-dimensional vectors of 0s, 
    #  this way those options won't contribute to the sum
    # [batch_size, max_length, D]    
    masked = torch.where(
        # we create an extra axis at the end of the tensor
        sequence_mask.unsqueeze(-1),  # this has shape [batch_size, max_length, 1]
        input_sequences,  # this has shape [batch_size, max_length, D]
        torch.zeros_like(input_sequences)  # this has shape [batch_size, max_length, D]
    )
    
    # we sum, along the sequence dimension (second last),
    #  the valid vectors (those that are not PAD)
    # we also divide by sequence length
    # [batch_size, D]
    avg = torch.sum(masked, dim=-2) / torch.sum(sequence_mask.float(), dim=-1, keepdims=True)

    return avg

In [ ]:
average_pooling(toy_emb(toy_batch), toy_batch != vocabulary.idx("-PAD-"))

<a name='ungraded-4'></a> **Ungraded Exercise 4 - Average pooling**

For an input batch with shape `[B, L, D]`, where `D` is the embedding dimension and `L` is max sequence length.

1. What's the output shape if we do average pooling over the sequence dimension?
2. Does average pooling require trainable parameters?


<details>
    <summary> <b>Click to see a solution</b> </summary>
 
1. `[B, D]` since average pooling will eliminate the sequence dimension, by elementwise average of the vectors in the sequence
2. No, it simply operates over the input vectors.
    
---
    
</details>      



<details>
    <summary> <b>Click to see a solution</b> </summary>

If you double-click the cell, you will be able to copy the code:

```python

# This checks the shape of the output
h = average_pooling(toy_emb(toy_batch), toy_batch != vocabulary.idx("-PAD-"))
assert h.shape == (toy_batch.shape[0], toy_emb_dim)

```

---
    
</details>      


<a name='ungraded-5'></a> **Ungraded Exercise 5 - Limitations of pooling**

Can you already recognise a big limitation of `sum_pooling` or `average_pooling` (in combination with one-hot or embeddings) as a means to represent a document?


<details>
    <summary> <b>Click to see a solution</b> </summary>
 
Sum or average pooling are not sensitive to the order in which the input vectors are presented. So, if the inputs to pooling do not preserve information about word order, then neither will the output. You can see that in our `toy_batch` the first and the third document are permutations of one another, hence they have the same BoW encoding and the same average embedding encoding.

Later, in this tutorial, we will address this limitation.
    
---
    
</details>      



<details>
    <summary> <b>Click to see a solution</b> </summary>

If you double-click the cell, you will be able to copy the code:

```python

bow = sum_pooling(F.one_hot(toy_batch, toy_vocab_size), toy_batch != vocabulary.idx("-PAD-"))
avgemb = average_pooling(toy_emb(toy_batch), toy_batch != vocabulary.idx("-PAD-"))
assert torch.all(bow[0] == bow[2])
assert torch.allclose(avgemb[0], avgemb[2])  # when comparing floating numbers, we use `allclose` rather than `all`

```

---
    
</details>      


<a name='sec:Mapping_from_one_real_coordinate_space_to_another'></a>
# Mapping from one real coordinate space to another


Sometimes we are working with vectors of a certain dimensionality $I$ and we need to convert them to vectors of another dimensionality $O$. This is very common, for example, when mapping a document encoding (e.g., average embedding, or BoW encoding) to $C$ logits.

<a name='sec:Linear_transformation'></a>
## Linear transformation 

A *linear transformation* is the standard way to get this done. 

If $\mathbf u \in \mathbb R^I$, then with $\mathbf W \in \mathbb R^{O\times I}$ and $\mathbf b \in \mathbb R^O$ we can use $\mathbf v = \mathbf W \mathbf u + \mathbf b$ to obtain $\mathbf v \in \mathbb R^O$.

In some situations, we prefer to use this transformation without a bias vector (i.e., setting $\mathbf b$ to a vector of $0$s); sometimes, this is also called a _projection_. 

Sometimes, we need to describe this operation in written form (for example, to sketch our model architecture). Here is a compact notation for it:
\begin{align}
    \mathbf v &= \mathrm{linear}_O(\mathbf u; \mathbf W, \mathbf b)
\end{align}


This is how we can construct a linear transformation with its trainable parameters

In [ ]:
toy_linear = nn.Linear(
    toy_emb_dim, # number of inputs
    3 # number of outputs
)

For such a toy example, you can inspect the initialised parameters:

In [ ]:
toy_linear.weight, toy_linear.bias

As expected, linear layers work on batched inputs too, so let's use the toy batch, convert its tokens to D-dimensional embeddings, combine all vectors in each sequence using average pooling, and then, finally, project each D-dimensional document encoding to 3 logits:

In [ ]:
toy_linear(average_pooling(toy_emb(toy_batch), toy_batch != vocabulary.idx("-PAD-")))

<a name='ungraded-6'></a> **Ungraded Exercise 6 - Linear transformation**

If we have a batch of inputs with shape `[B, L]`, use a `D`-dimensional embedding layer to encode tokens,  average pooling to encode documents, and then a linear transformation to `K`-dimensional outputs:

1. What's the shape of the output?
2. How many trainable parameters are necessary to specify the linear layer?


<details>
    <summary> <b>Click to see a solution</b> </summary>
 
1. After the embedding layer we have `[B, L, D]`, after the pooling operation we have `[B, D]` and after the linear transformation we have `[B, K]`.
2. We need to store a matrix of shape `[K, D]` and a vector of `K` biases to linearly transform `D`-dimensional vectors into `K`-dimensional vectors.
    
---
    
</details>      


<a name='sec:Nonlinear_activation_functions'></a>
## Nonlinear activation functions

Sometimes we need to work on a _subspace_ of the real coordinate space, for example, where numbers are constrained to being positive, or strictly positive, or strictly positive and sum up to 1, etc.

We can achieve this by working with activation functions. An activation function will not change the dimensionality of its input, and it does not require any trainable parameter, typically, an activation function is a formula that transforms a vector elementwise. 

For example, if $\mathbf u \in \mathbb R^D$
* $\exp(\mathbf u)$ will apply $\exp(u_d)$ to each coordinate $u_d$ of $\mathbf u$, hence returning a vector of $D$ strictly positive numbers;
* $\mathrm{relu}(\mathbf u)$ will apply $\max(0, u_d)$ to each coordinate $u_d$ of $\mathbf u$, hence returning a vector of $D$ positive (and possibly 0) numbers;
* $\tanh(\mathbf u)$ applies $\tanh(u_d)$ to each coordinate $u_d$ of $\mathbf u$, hence return a vector $D$ numbers in the space $(-1, 1)$

In [ ]:
torch.exp(torch.tensor([-2.5, -1., 0., 1, 2.5]))

In [ ]:
torch.relu(torch.tensor([-2.5, -1., 0., 1, 2.5]))

In [ ]:
torch.tanh(torch.tensor([-2.5, -1., 0., 1, 2.5]))

Of course, these operations also work with batches:

In [ ]:
torch.exp(torch.tensor([[-2.5, -1., 0., 1, 2.5], [-3., -2., -1, 0.5, 1.5]]))

<a name='ungraded-7'></a> **Ungraded Exercise 7 - Activation functions**

Play a bit with the following activations and explain what they do:
1. `torch.sigmoid`
2. `F.softplus`
3. `F.softmax`


<details>
    <summary> <b>Click to see a solution</b> </summary>

If you double-click the cell, you will be able to copy the code:

```python

# sigmoid maps from R to (0, 1), it's appropriate to predict probability values
torch.sigmoid(torch.tensor([[-2.5, -1., 0., 1, 2.5], [-3., -2., -1, 0.5, 1.5]]))

```

---
    
</details>      



<details>
    <summary> <b>Click to see a solution</b> </summary>

If you double-click the cell, you will be able to copy the code:

```python

# softplus maps from R to R+, it's appropriate to predict statistical parameters that must be strictly positive
# like rate, scale, concentration, etc.
F.softplus(torch.tensor([[-2.5, -1., 0., 1, 2.5], [-3., -2., -1, 0.5, 1.5]]))

```

---
    
</details>      



<details>
    <summary> <b>Click to see a solution</b> </summary>

If you double-click the cell, you will be able to copy the code:

```python

# softmax maps from R^K to Simplex K-1, it's appropriate to predict probability vectors (that are normalised as to sum to 1)
print(F.softmax(torch.tensor([[-2.5, -1., 0., 1, 2.5], [-3., -2., -1, 0.5, 1.5]]), dim=-1))
assert torch.allclose(torch.sum(F.softmax(torch.tensor([[-2.5, -1., 0., 1, 2.5], [-3., -2., -1, 0.5, 1.5]]), dim=-1), -1), torch.ones(2))

```

---
    
</details>      


<a name='sec:Composing_multiple_vectors'></a>
# Composing multiple vectors

We now look into *composing* multiple vectors of fixed dimensionality into one (or more) vectors, possibly changing the dimensionality of the output (with respect to the dimensionality of the input(s)), possibly preserving information about their relative order.

<a name='sec:Concatenation'></a>
## Concatenation

The simplest thing we can do is to concatenate vectors, in their given order. This operation does not require any trainable parameters, but it has a downside: the size of the output increases as a function of the number of inputs.

For example, we can concatenate an $A$-dimensional vector $\mathbf u$ with a $B$-dimensional vector $\mathbf v$ obtaining a $A+B$-dimensional vector:
\begin{align}
    \mathbf h &= \mathrm{concat}(\mathbf u, \mathbf v) = (u_1, \ldots, u_{A}, v_1, \ldots, v_{B})^\top
\end{align}

<a name='ungraded-8'></a> **Ungraded Exercise 8 - Concatenation**

Concatenate the BoW encoding of the documents in the toy batch with their average embedding.


<details>
    <summary> <b>Click to see a solution</b> </summary>

If you double-click the cell, you will be able to copy the code:

```python

bow = sum_pooling(F.one_hot(toy_batch, toy_vocab_size), toy_batch != vocabulary.idx("-PAD-"))
avgemb = average_pooling(toy_emb(toy_batch), toy_batch != vocabulary.idx("-PAD-"))
bow_avg = torch.cat([bow, avgemb], -1)
assert bow_avg.shape[-1] == toy_vocab_size + toy_emb_dim

```

---
    
</details>      


<a name='sec:Feed_forward_network'></a>
## Feed forward network


A basic feed-forward network (FFNN) combines two linear transformations with a non-linear activation function in between. It is also possible to have deep FFNN, by stacking additional linear transformations (and nonlinearities).

This is the basic one:

\begin{align}
    \mathbf h &= a(\mathrm{linear}_H(\mathbf u; \theta_{\text{hid}})) \\
    \mathbf o &= \mathrm{linear}_O(\mathbf h; \theta_{\text{out}}) \\
\end{align}

where $a(\cdot)$ is an elementwise nonlinearity (typically, $\tanh$ or $\mathrm{relu}$), $\theta_{\text{hid}}$ are the parameters of the first input-to-hidden layer (a weight matrix and a bias vector) and  $\theta_{\text{out}}$ are the parameters of the second hidden-to-output layer (another weight matrix and another bias vector). See that instead of writing down the parameters explicitly, we named them (using $\theta$ and a suggestive subscript); this is a convenient notation shortcut.

A nice way to prescribe FFNNs in torch is to use the so-called `Sequential` API, which stacks transformations:

In the following example, we concatenate two views of each document: the bag-of-words encoding and the average token embedding.

We then map each such vector to 7 ReLU hidden units and then to 3 output units.

In [ ]:
hidden_size = 7
output_size = 3
toy_ffnn = nn.Sequential(
    nn.Linear(toy_vocab_size + toy_emb_dim, hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, output_size)
)
assert num_parameters(toy_ffnn) == ((toy_vocab_size + toy_emb_dim)*hidden_size + hidden_size + hidden_size*output_size + output_size)

As expected, a FFNN accepts batched inputs:

In [ ]:
bow = sum_pooling(F.one_hot(toy_batch, toy_vocab_size), toy_batch != vocabulary.idx("-PAD-"))
avgemb = average_pooling(toy_emb(toy_batch), toy_batch != vocabulary.idx("-PAD-"))
bow_avg = torch.cat([bow, avgemb], -1)
assert toy_ffnn(bow_avg).shape == (bow_avg.shape[0], output_size)
toy_ffnn(bow_avg)

<a name='ungraded-9'></a> **Ungraded Exercise 9 - FFNN**

Use the Sequential API to specify a FFNN similar to the example above, but with 2 hidden layers: the first with 7 units and the second with 13 units. Use ReLU for the first hidden layer and Tanh for the second hidden layer. The output of your FFNN should have 3 units, like in the example above. Inspect the number of parameters of the layer. Then, test it on the concatenation of bow and avg encodings, as we did above. Explain why the output shape is the same as in the example above.


<details>
    <summary> <b>Click to see a solution</b> </summary>

If you double-click the cell, you will be able to copy the code:

```python

hidden_sizes = [7, 13]
output_size = 3
toy_ffnn2 = nn.Sequential(
    nn.Linear(toy_vocab_size + toy_emb_dim, hidden_sizes[0]),
    nn.ReLU(),
    nn.Linear(hidden_sizes[0], hidden_sizes[1]),
    nn.Tanh(),
    nn.Linear(hidden_sizes[1], output_size)
)
assert num_parameters(toy_ffnn2) == ((toy_vocab_size + toy_emb_dim)*hidden_sizes[0] + hidden_sizes[0] + hidden_sizes[0]*hidden_sizes[1] + hidden_sizes[1] + hidden_sizes[1]*output_size + output_size)

assert toy_ffnn2(bow_avg).shape == (bow_avg.shape[0], output_size)

assert toy_ffnn2(bow_avg).shape == toy_ffnn(bow_avg).shape

# The output shape is the same as before 
# because the shape is determined by the ouptut_size 
# and not by the number of hidden layers
toy_ffnn2(bow_avg)

```

---
    
</details>      


<a name='sec:Recurrent_neural_network_encoder'></a>
## Recurrent neural network encoder


The **RNN encoder** uses one or more feed-forward nets inside of a for loop to iterate over the steps of a sequence, at each step the RNN encoder combines a _recurrent_ state and the input at that step using a FFNN. Rather than one different FFNN per step, the RNN encoder reuses the same FFNN (that's where the _recurrent_ in its name comes from). Suppose we have a sequence of vectors $\mathbf e_{1:l}$, for example, those are the the token embeddings for a document $w_{1:l}$.

The RNN encoder has a initial state $\mathbf u_0$, this could be a vector of 0s, for example. Then, at each step $i \in [l]$ of the sequence, the RNN encoder combines the recurrent state $u_{i-1}$ (as obtained in the step before) with the current input $\mathbf e_i$, this combination produces a new value $\mathbf u_i$ for the recurrent state, which will be used in the next step. 

See the figure for an illustration:
<img src="https://raw.githubusercontent.com/probabll/ntmi-tutorials/main/img/rnn-encoder.jpeg" alt="drawing" width="500"/>

We can describe the computation as follows:
\begin{align}
\mathbf u_i &= \mathrm{rnnstep}_K(\mathbf u_{i-1}, \underline{\mathbf e_{i}}; \theta_{\text{enc}}) \quad \text{for }i \in [l].
\end{align}

The simplest implementation of the block `rnnstep` looks like this:
\begin{align}
\mathrm{rnnstep}_K(\mathbf u_{i-1}, \underline{\mathbf e_{i}}; \theta_{\text{enc}}) &= \tanh(\mathbf U \mathbf u_{i-1} + \mathbf W \mathbf e_{i-1}) 
\end{align}
where the trainable parameters $\theta_{\text{enc}} = \{\mathbf U \in \mathbb R^{K\times K}, \mathbf W \in \mathbb R^{K\times D}, \}$ are two matrices that project the recurrent state and the current input to size $K$.


If you pay close attention to the illustration, or to the formulae, you will see that the recurrent state at any one position $i$ can potentially store information from any of the inputs $\mathbf e_1, \ldots, \mathbf e_{i-1}$. Besides, due to the nonlinearity of the rnnstep function, the state $\mathbf u_i$ is sensitive to the order in which the inputs are presented (that is, if we presented inputs in a different order, the numerical values of the coordinates of $\mathbf u_i$ might differ). This is an important aspet of a feature function for natural language processing, given that natural languages encode a lot of information in word order.

In written form, the RNN encoder can be denoted even more compactly as 
\begin{align}
\mathbf u_{1:l} &= \mathrm{rnnenc}_K(\mathbf e_{1:l}; \theta_{\text{enc}})
\end{align}
it takes a sequence of vectors as input and returns a sequence of $K$-dimensional vectors as output. The last vector  $\mathbf u_l$ in the output sequence has information about the entire document $x$ in order.






### LSTM

In this tutorial we will use a modern architecture for the recurrent cell called [*Long Short-Term Memory*](https://arxiv.org/pdf/1503.04069.pdf) (LSTM for short). [It's already implemented for us in torch](https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html).

You do not need to study the LSTM paper, what you will need to know will be explained in this tutorial. 

Here we briefly explain the **internal design of the LSTM**. The choice of letters we use in this part are internal to the LSTM and are not to be confused for letters used in other contexts.

For a step $t$, let $\mathbf x_t$ be an $I$-dimensional input to an LSTM (at this point it's not important whether this corresponds to a step in the token sequence of in the tag sequence, or any sequence really, it just matters that at some point in time $t$ we want to use this input to udpate the LSTM memory).

At this point, the memory of an LSTM is made of two $K$-dimensional vectors called the *cell vector* $\mathbf c_{t-1}$ and the *hidden state* $\mathbf h_{t-1}$, each of which is $K$-dimensional. When we process the input $\mathbf x_t$ with an LSTM, these two vectors are updated step by step as shown below:

\begin{align}
    \mathbf i_t &=\mathrm{sigmoid}(\mathrm{affine}_K(\mathbf x_t; \theta_1) + \mathrm{affine}_K(\mathbf h_{t-1}; \theta_2))\\
    \mathbf f_t &=\mathrm{sigmoid}(\mathrm{affine}_K(\mathbf x_t; \theta_3) + \mathrm{affine}_K(\mathbf h_{t-1}; \theta_4))\\
    \mathbf g_t &=\tanh(\mathrm{affine}_K(\mathbf x_t; \theta_5) + \mathrm{affine}_K(\mathbf h_{t-1}; \theta_6))\\
    \mathbf o_t&=\mathrm{sigmoid}(\mathrm{affine}_K(\mathbf x_t; \theta_7) + \mathrm{affine}_K(\mathbf h_{t-1}; \theta_8))\\
    \mathbf c_t &= \mathbf f_t \odot \mathbf c_{t-1} + \mathbf i_t \odot \mathbf g_t \\
    \mathbf h_t &= \mathbf o_t \odot \tanh(\mathbf c_t)
\end{align}

The first four steps compute the following using the input and the hidden state: the *input gate* $\mathbf i_t$, then the *forget gate* $\mathbf f_t$, the *draft cell* $\mathbf g_t$, and the *output gate* $\mathbf o_t$. 
These are all $K$-dimensional, and the affine transformations all have their own parameters (there 8 such affine transformations in total, they map either from $I$ dimensions to $K$ dimensions, or from $K$ dimensions to $K$ dimensions, and they have biases vectors in them). The last couple of steps finally uppdate the cell and the hidden state by combining the intermediate gates and draft cell. The symbol $\odot$ denotes elementwise multiplication. After the update the LSTM memory is made of two states $\mathbf c_t$ and $\mathbf h_t$, which are $K$-dimensional. Typically, the cell is internal to the LSTM and we rarely need to use it for anything outside of it. It is the hidden state after each update that we normally want to use in applications.  The torch implementation gives us access to both of them, and we will see later how to use it.

Compared to our illustration of the RNN encoder, we will have token embeddings as input, and we will interpret hidden states as outputs of the LSTM. 

---

In [ ]:
toy_hidden_size = 6
toy_lstm = nn.LSTM(
    input_size=toy_emb_dim, # size of the vectors in the input sequence
    hidden_size=toy_hidden_size, # size of the recurrent cell
    num_layers=1,
    batch_first=True,  # this is important, it's telling the nn.LSTM class
    bidirectional=False,  # we will explain this argument in the next example
)
toy_lstm

In [ ]:
num_parameters(toy_lstm)

In [ ]:
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

In [ ]:
# [batch_size, max_len, emb_dim]
e = toy_emb(toy_batch)
# [batch_size, max_len, hidden_dim]
# internally, the LSTM maintains two vectors in the memory
# the forward method of the LSTM class will return 
# a tensor which has the sequence of so called hidden states (this is usually what you want to use in a text encoder)
# and tuple of tensors that can be used in case you need access to the internal 
# mechanism of the LSTM cell

# For convenient, torch provides two auxiliary functions that help us deal with 
# batches of sequences that may differ in length
# First, we pack the padded sequences in a special way using `pack_padded_sequence`
#  for this to work correctly, torch needs to know the length of the sequences (discounting padding)
# [batch_size]
lengths = (toy_batch != vocabulary.idx("-PAD-")).long().sum(-1)
packed_seqs = pack_padded_sequence(e, lengths.cpu(), batch_first=True, enforce_sorted=False)        
# it's important to tell torch that the first axis of our tensors is for the batch (with `batch_first=True`)
# it's also important to tell torch that our sequences are _not_ sorted by length (with `enfore_sorted=False`)

# Next, we run the LSTM on packed sequences,
#  this returns, for every sequence, the states for all steps and a tuple (final state, final cell)
h, (last_h, last_c) = toy_lstm(packed_seqs)

# Finally, before going ahead and using `h`, we call `pad_packed_sequence`
#  this returns the tensor of states with padding positions zeroed out
h, _ = pad_packed_sequence(h, batch_first=True)

assert h.shape == toy_batch.shape + (toy_hidden_size,)

In [ ]:
print(h)  # se how the state information is zeroed out for the -PAD- positions
# this is because we used torch's helper functions (pack_padded_sequence and pad_packed_sequence)

In [ ]:
print(last_h) # this tensor, returned by the LSTM class gives us convenient access to the last state of each sequence

If we wanted to combine the LSTM outputs into a single vector, we could use average pooling, for example. This does not destroy the infromation about word order, since that information is already coded in the LSTM outputs.
Alternatively, we could use the last state of the sequence. This is a practical choice, and there's no theory to support one option or the other.

<a name='sec:Bidirectional_RNN_encoder'></a>
### Bidirectional RNN encoder

In the previous section, when encoding a document $x$, we processed the words from left-to-right. In reality, there's no need to limit ourselves that order. Rather, we have access to the document anyway, so why not also encode it from right-to-left. This way, whenever we look at a given position $i$, we can obtain information from the left (i.e., from $\mathbf e_1, \ldots, \mathbf e_i$) and from the right (i.e., from $\mathbf e_i, \ldots, \mathbf e_l$). This can be useful in certain tasks.  

An RNN cell, by design, makes computations in a single direction (e.g., left-to-right), but we can use 2 different RNN cells, one that reads the sequence in one order and another that reads the sequence in reversed order.

We don't need to invent a new RNN cell for this, we can simply reverse the inputs to a standard RNN cell:
\begin{align}
\mathbf r_{1:l} &= \mathrm{reverse}(\mathbf e_{1:l}) \\
\mathbf v_i &= \mathrm{rnnstep}_K(\mathbf v_{i-1}, \mathbf r_{i}; \theta_{\text{renc}}) ~.
\end{align}
Because the inputs have been reversed in order. For example, in a sentence of length $l=10$, $\mathbf v_2$ knows about $w_{9}$ through $\mathbf r_2$ and about $w_{>9}$ through $\mathbf v_{1}$. The last state $\mathbf v_l$ has information about the entire document $w_{1:l}$, but processed it in reversed order.

Therefore a reversed RNN encoder can be denoted compactly as follows:
\begin{align}
\mathbf v_{1:l} &= \mathrm{rnnenc}_K(\mathrm{reverse}(\mathbf e_{1:l}); \theta_{\text{renc}})
\end{align}


Note that we named the parameter set differently: $\theta_{\text{enc}}$ for the first RNN cell, and $\theta_{\text{renc}}$ for the second one, that's because we indeed want to have two different sets of parameters. If we used the same set of parameters for both directions, that probably would not work very well, as reading in one direction and reading in another are conceptually two different operations.

The **bidirectional RNN encoder** is our prefered text encoder, it can be denoted as follows:
\begin{align}
\mathbf o_{1:l} &= \mathrm{birnn}_{2K}(\mathbf e_{1:l}; \theta_{\text{enc}} \cup \theta_{\text{renc}})
\end{align}
and here are the operations that it performs:
\begin{align}
\mathbf u_{1:l} &= \mathrm{rnnenc}_K(\mathbf e_{1:l}; \theta_{\text{enc}})\\
\mathbf v_{1:l} &= \mathrm{rnnenc}_K(\mathrm{reverse}(\mathbf e_{1:l}); \theta_{\text{renc}})\\
\mathbf o_{i} &= \mathrm{concat}(\mathbf u_i, \mathbf v_{l-i+1}) & \text{for }i \in \{1, \ldots, l\}
\end{align}

Its outputs are $2K$-dimensional because after processing the sequence from left-to-right with the first RNN encoder and from right-to-left with the second RNN encoder, it then concatenates the two views of the process in such a way that $\mathbf o_i$ has information about $w_i$, $w_{<i}$ and $w_{>i}$.

See the figure as an illustration of how the two RNN cells can be used to obtain the bidirectional RNN encoder: 

<img src="https://raw.githubusercontent.com/probabll/ntmi-tutorials/main/img/birnn.jpeg" alt="drawing" width="500"/>


Luckily, `nn.LSTM` implements all that for us, and we don't really need to worry about reversing anything ourserlves. See the example below.

In [ ]:
toy_bilstm = nn.LSTM(
    input_size=toy_emb_dim,
    hidden_size=toy_hidden_size,
    num_layers=1,
    batch_first=True,
    bidirectional=True,  # now we employ one LSTM for each direction
)
toy_bilstm

In [ ]:
num_parameters(toy_bilstm)

In [ ]:
assert num_parameters(toy_bilstm) == 2*num_parameters(toy_lstm), "A BiLSTM is made of two LSTMs"

In [ ]:
# [batch_size, max_len, 2*hidden_dim]
# as for the standard LSTM, we only use the first of its outputs, namely, 
# a tensor of states, this time the states are concatenated for two directions, 
# thus they will be twice as large

# As before, we use torch's helper functions to correctly deal with the variable length
# of our padded sequences
packed_seqs = pack_padded_sequence(e, lengths.cpu(), batch_first=True, enforce_sorted=False)        
h2, (last_h2, last_c2) = toy_bilstm(packed_seqs)
h2, _ = pad_packed_sequence(h2, batch_first=True)
assert h2.shape == toy_batch.shape + (2*toy_hidden_size,)  # BiLSTM outputs are the concatenation of 2 LSTM outputs

In [ ]:
h2.shape # the shape here is as expected [batch_size, max_len, 2*hidden_dim]

In [ ]:
last_h2.shape  # the shape here is a bit different: [num_layers, batch_size, hidden_dim]
# where each direction counts as 1 layer, hence num_layers is 2

In [ ]:
# to concatenate the final states of each sequence
# we can first move the first axis (num_layers) to the end of the tensor using `permute`
# and then flatten the last two axis (hidden_dim, num_layers)
# obtaining output shape [batch_size, num_layers * hidden_dim]
torch.flatten(torch.permute(last_h2, (1, 2, 0)), 1, 2).shape